In [1]:
from instructor import from_openai, Mode
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv
import os
os.environ["OPENAI_LOG"] = "debug"   # must be set before the first client is built

from my_agent.schemas.spanish_conj_model import SpanishVerbConjugation

load_dotenv()

client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"), max_retries=3)
instructor_llm = from_openai(client=client, model="gpt-4o-mini", temperature=0.2, mode=Mode.JSON)



# Define hook functions
def log_kwargs(**kwargs):
    print(f"Function called with kwargs: {kwargs}")


def log_exception(exception: Exception):
    print(f"An exception occurred: {str(exception)}")

    
instructor_llm.on("completion:kwargs", log_kwargs)
instructor_llm.on("completion:error", log_exception)

# x = await instructor_llm.create(
#     messages=[
#         {"role": "system", "content": "You are an expert Spanish linguist. "
#                     "Return ONLY valid JSON matching the schema and do it fast."},
#         {"role": "user", "content": f"Conjugate the verb 'comprar'."}
#     ],
#     response_model=SpanishVerbConjugation,
# )

In [ ]:
import json
print(json.dumps(x.model_dump(), indent=4))

In [ ]:
import dotenv
from my_agent.tools.media_tool import fetch_photos
from my_agent.schemas.visual_model import Media
load_dotenv()
from pydantic import BaseModel
from pydantic import Field

def get_media(source_word: str, target_word: str, source_language: str, target_language: str) -> Media:
    """
    Get the most memorable photo for a vocabulary word.
    Steps:
    1. Translate word to English with descriptors
    2. Fetch Pexels photos
    3. Ask LLM to choose the best one
    """

    class Criteria(BaseModel):
        query: str = Field(description="English search query plus two descriptive synonyms to find the most relevant photos in Pexels")

    try:
        # translate to english and get criteria
        translation_and_choice_prompt = f"""
        Given the {source_language} word '{source_word}', respond with:
        1. `query` – an English search query plus a few descriptive synonyms to find the most relevant photos in Pexels.

        Respond in the JSON format provided.
        """
        result = instructor_llm.create(response_model=Criteria, messages=[{"role": "user", "content": translation_and_choice_prompt}])


        # fetch three photos
        photos = fetch_photos(result.query, per_page=15)

        print(photos)


        # choose the best photo
        rank_prompt = f"""Find the most adquate photo for the following criteria that depicts best the {source_language} word '{source_word}' that makes it easy to remember the {target_language} word '{target_word}'.

        Photos:
        {[p.model_dump() for p in photos]}

        Respond only in the JSON format provided.
        """

        print(rank_prompt)

        result = instructor_llm.create(
            response_model=Media,
            messages=[{"role": "user", "content": rank_prompt}],
        )

        print(result)
            
    except Exception as e:
        return f"Error generating visual media: {e!s}"

get_media(source_word="Kaputze", target_word="gorra", source_language="Deutsch", target_language="Spanish")

In [8]:
# from my_agent.tools.pronounciation_tool import LANGUAGE_VOICES
from elevenlabs import VoiceSettings, play, save
from dotenv import load_dotenv
load_dotenv()

from elevenlabs.client import ElevenLabs

LANGUAGE_VOICES = {
    "English": "EXAVITQu4vr4xnSDxMaL",  # Bella - English
    "Spanish": "94zOad0g7T7K4oa7zhDq",  # Mauricio
    "German": "XrExE9yKIg1WjnnlVkGX",   # Matilda - Multi-language
}

target_language = "Spanish"
voice_id = LANGUAGE_VOICES.get(target_language, LANGUAGE_VOICES["English"])
        
client = ElevenLabs()

audio = client.text_to_speech.convert(
    text="Hello",
    voice_id=voice_id,
    language_code="es",
    model_id="eleven_flash_v2_5",
    output_format="mp3_44100_128",
    voice_settings=VoiceSettings(
        speed=0.7,
        stability=0.3,
        similarity_boost=0.2,
        style=0.2,
        use_speaker_boost=True
    )
)

# play(audio=audio)
save(audio, "quisquilloso.mp3")